In [412]:
import os
import sys
import time
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import cv2

In [402]:
# ASCII characters used to represent the grayscale intensity
ASCII_CHARS_REV = "#@*?-,. "
# ASCII_CHARS_REV = "HelloWorld! "
ASCII_CHARS = ""
for char in reversed(ASCII_CHARS_REV):
    ASCII_CHARS += char

In [403]:
def detect_cont_string(ascii_line_str):
    """
    Returns the positions of a continous non-blank string.

    Args:
        ascii_line_str (str): Single line of an ascii art str

    Returns:
        (List[List[int, int]]): Start and end position of the continous non-blank string.
    """

    str_pos_lis = []
    for i, c in enumerate(ascii_line_str):
        if c != " ":
            str_pos_lis.append(i)
    
    if len(str_pos_lis) == 0:
        return str_pos_lis
    
    str_grp_lis = [[str_pos_lis[0]]]
    for i, v in enumerate(str_pos_lis):
        if i != 0:
            if v - 1 != str_pos_lis[i-1]:
                str_grp_lis[-1].append(str_pos_lis[i-1])
                str_grp_lis.append([v])
    
    str_grp_lis[-1].append(str_pos_lis[-1])
    
    return str_grp_lis

In [421]:
def image_to_ascii(image, width:int=80):
    """
    Convert an image to ASCII characters.

    Args:
        image (ImageFile): PIL Image object
        width (int): Desired output width

    Returns:
        (str): ASCII art as string
    """
    # Calculate aspect ratio and set new height
    aspect_ratio = image.height / image.width
    new_height = int(aspect_ratio * width * 0.5)  # Adjusting for font aspect ratio

    # Resize the image
    image = image.resize((width, new_height))
    
    # Convert to grayscale
    gray_image = image.convert("L")
    # display(gray_image)

    # Convert pixels to ASCII
    pixels = np.array(gray_image)
    max_pixel = max([max(row) for row in pixels])
    min_pixel = min([min(row) for row in pixels])
    sep = int((max_pixel - min_pixel)/len(ASCII_CHARS))
    # print(sep)

    # ascii_str = ""
    # for row in pixels:
    #     ascii_str += "".join(ASCII_CHARS[min(pixel // sep, pixel // 40)] for pixel in row) + "\n"

    introduction_str = "hello_fellow_data_scientis__i_am_pradyuman__"
    len_introduction_str = len(introduction_str)
    ascii_str = ""
    for row in pixels:
        cur_introduction_str = ""; str_to_add = ""
        for pixel in row:
            str_to_add += ASCII_CHARS[min(pixel // sep, pixel // 40)]
        
        str_grp_lis = detect_cont_string(ascii_line_str = str_to_add)
        prev_end = 0
        if len(str_grp_lis) != 0:
            for grp in str_grp_lis:
                cur_introduction_str += " "*(grp[0] - prev_end)
                prev_end = grp[1]

                ratio_nbs_intro_str = (grp[1] - grp[0])/len_introduction_str
                cur_introduction_str += int(ratio_nbs_intro_str)*introduction_str

                remainder = ratio_nbs_intro_str - int(ratio_nbs_intro_str)
                cur_introduction_str += introduction_str[:int(len_introduction_str*remainder)]

                introduction_str = introduction_str[int(len_introduction_str*remainder):] + introduction_str[:int(len_introduction_str*remainder)]
            cur_introduction_str += " "*(len(str_to_add) - str_grp_lis[-1][-1])
            str_to_add = cur_introduction_str



            # ratio_nbs_intro_str = (cont_nbs_pos[1] - cont_nbs_pos[0])/len_introduction_str
            # cur_introduction_str += int(ratio_nbs_intro_str)*introduction_str
            # remainder = ratio_nbs_intro_str - int(ratio_nbs_intro_str)
            # cur_introduction_str += introduction_str[:int(len_introduction_str*remainder)]
            # introduction_str = introduction_str[int(len_introduction_str*remainder):] + introduction_str[:int(len_introduction_str*remainder)]

            # str_to_add = " "*len(str_to_add[:cont_nbs_pos[0]]) + cur_introduction_str + " "*len(str_to_add[cont_nbs_pos[1]:])
                
        ascii_str += "".join(str_to_add) + "\n"
    
    
    

    return ascii_str

In [405]:
def extract_frames_from_gif(gif_path):
    """
    Extract frames from a GIF and return them as a list of PIL images.
    
    Args:
        gif_path (str): Path to the GIF file

    Return:
        (List[ImageFile]): List of PIL Image objects
    """
    frames = []
    image = Image.open(gif_path)

    try:
        while True:
            frame = image.convert("RGB")
            frames.append(frame)
            image.seek(image.tell() + 1)  # Move to next frame
    except EOFError:
        pass

    return frames

In [485]:
# def ascii_to_image(ascii_text, font_path=None, img_size=(800, 600)):
#     """
#     Convert ASCII text to an image with proper monospaced alignment.
#     :param ascii_text: ASCII text to be converted
#     :param font_path: Path to a monospaced font (optional)
#     :param img_size: Tuple (width, height) for the output image
#     :return: PIL Image
#     """
#     font_size = 12  # Adjust for readability

#     # Ensure a monospaced font is used
#     if font_path and os.path.exists(font_path):
#         font = ImageFont.truetype(font_path, font_size)
#     else:
#         font = ImageFont.load_default()  # Default may not be monospaced

#     # Determine the size of a single character
#     char_width, char_height = font.getsize("A")

#     # Calculate dynamic image size based on ASCII text dimensions
#     lines = ascii_text.split("\n")
#     max_line_length = max(len(line) for line in lines)
#     img_width = char_width * max_line_length + 10  # Add some padding
#     img_height = char_height * len(lines) + 10  # Adjust for lines

#     # Create a new black image
#     img = Image.new("RGB", (img_width, img_height), "black")
#     draw = ImageDraw.Draw(img)

#     # Draw each character at fixed spacing
#     x_offset = 5  # Left padding
#     y_offset = 5  # Top padding

#     for i, line in enumerate(lines):
#         for j, char in enumerate(line):
#             draw.text((x_offset + j * char_width, y_offset + i * char_height), char, font=font, fill="white")

#     return img


In [487]:
def ascii_to_image(ascii_text, font_path=None, img_size=(800, 600)):
    """
    Convert an ASCII string to a PIL image.
    :param ascii_text: ASCII text to be converted
    :param font_path: Optional path to a font file
    :param img_size: Size of the output image
    :return: PIL Image
    """
    font_size = 10  # Adjust font size
    font = ImageFont.load_default()  # Use default font

    if font_path and os.path.exists(font_path):
        font = ImageFont.truetype(font_path, font_size)

    img = Image.new("RGB", img_size, "black")
    draw = ImageDraw.Draw(img)

    # Split the ASCII text into lines
    lines = ascii_text.split("\n")

    y = 5  # Start drawing from the top
    for line in lines:
        draw.text((5, y), line, font=font, fill="white")
        y += font_size  # Move down for next line

    return img

In [488]:
def save_ascii_gif(ascii_frames, output_path="ascii_animation.gif", duration=100, font_path=None):
    """
    Convert a list of ASCII frames to a GIF.
    :param ascii_frames: List of ASCII strings
    :param output_path: Output file path for the GIF
    :param duration: Duration per frame in milliseconds
    :param font_path: Optional font path
    """
    images = [ascii_to_image(frame, font_path) for frame in ascii_frames]
    
    # Save as GIF
    images[0].save(output_path, save_all=True, append_images=images[1:], duration=duration, loop=0)
    print(f"GIF saved as {output_path}")

In [489]:
gif_path = "../data/star.gif"

# image = Image.open(gif_path)
frames = extract_frames_from_gif(
    gif_path=gif_path
)

ascii_frames = []
for f in frames:
    ascii_frames.append(
        image_to_ascii(image=f)
    )

In [490]:
# print(
#     image_to_ascii(
#         image = image
#     )
# )

In [491]:
save_ascii_gif(ascii_frames, output_path="../data/ascii_animation.gif", duration=1)

GIF saved as ../data/ascii_animation.gif


In [469]:
for f in ascii_frames:
    print(f)